In [1]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("cs544")
         .config("spark.sql.shuffle.partitions", 10)
         .config("spark.ui.showConsoleProgress", False)
         .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.2')
         .getOrCreate())

df = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("subscribe", "stations-json")
    .option("startingOffsets", "earliest")
    .load()
)

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bea2b15a-528e-4f1d-89aa-4d8ba894dfa2;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.2 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.ap

In [2]:
from pyspark.sql.functions import col, from_json
data = spark.readStream(

[('key', 'binary'),
 ('value', 'binary'),
 ('topic', 'string'),
 ('partition', 'int'),
 ('offset', 'bigint'),
 ('timestamp', 'timestamp'),
 ('timestampType', 'int')]

In [7]:
schema = "station STRING, date DATE, degrees DOUBLE, raining INTEGER"
data = (df.select(col("key").cast("string"),
          from_json(col("value").cast("string"), schema).alias("value")))
q = (data.writeStream
 .format("console").trigger(processingTime="5 seconds")
 .outputMode("append").start())

23/04/29 00:16:36 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-df8559fc-5fe8-491f-91e6-5915cb95d69d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/04/29 00:16:36 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+--------------------+
|key|               value|
+---+--------------------+
|  F|{F, 2000-01-01, 4...|
|  I|{I, 2000-01-01, 4...|
|  J|{J, 2000-01-01, 1...|
|  F|{F, 2000-01-02, 4...|
|  I|{I, 2000-01-02, 3...|
|  J|{J, 2000-01-02, 2...|
|  F|{F, 2000-01-03, 4...|
|  I|{I, 2000-01-03, 3...|
|  J|{J, 2000-01-03, 1...|
|  F|{F, 2000-01-04, 3...|
|  I|{I, 2000-01-04, 3...|
|  J|{J, 2000-01-04, 1...|
|  F|{F, 2000-01-05, 4...|
|  I|{I, 2000-01-05, 3...|
|  J|{J, 2000-01-05, 1...|
|  F|{F, 2000-01-06, 4...|
|  I|{I, 2000-01-06, 2...|
|  J|{J, 2000-01-06, 1...|
|  F|{F, 2000-01-07, 3...|
|  I|{I, 2000-01-07, 3...|
+---+--------------------+
only showing top 20 rows



23/04/29 00:16:50 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 13425 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+---+--------------------+
|key|               value|
+---+--------------------+
|  F|{F, 2008-09-01, 1...|
|  I|{I, 2008-09-01, 1...|
|  J|{J, 2008-09-01, 5...|
|  F|{F, 2008-09-02, 1...|
|  I|{I, 2008-09-02, 1...|
|  J|{J, 2008-09-02, 6...|
|  F|{F, 2008-09-03, 1...|
|  I|{I, 2008-09-03, 9...|
|  J|{J, 2008-09-03, 7...|
|  F|{F, 2008-09-04, 1...|
|  I|{I, 2008-09-04, 8...|
|  J|{J, 2008-09-04, 6...|
|  F|{F, 2008-09-05, 1...|
|  I|{I, 2008-09-05, 9...|
|  J|{J, 2008-09-05, 5...|
|  F|{F, 2008-09-06, 1...|
|  I|{I, 2008-09-06, 8...|
|  J|{J, 2008-09-06, 6...|
|  F|{F, 2008-09-07, 9...|
|  I|{I, 2008-09-07, 9...|
+---+--------------------+
only showing top 20 rows



In [8]:
q.stop()

In [5]:
spark.streams.active[0].stop()

In [6]:
counts_df = df.groupBy("station").count()
s = counts_df.writeStream.format("console").trigger(processingTime="5 seconds").outputMode("complete").start()
s.awaitTermination(30)
s.stop()

AnalysisException: cannot resolve 'station' given input columns: [key, offset, partition, timestamp, timestampType, topic, value];
'Aggregate ['station], ['station, count(1) AS count#35L]
+- StreamingRelationV2 org.apache.spark.sql.kafka010.KafkaSourceProvider@3c490dd4, kafka, org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaTable@7a654478, [startingOffsets=earliest, kafka.bootstrap.servers=kafka:9092, subscribe=stations-json], [key#7, value#8, topic#9, partition#10, offset#11L, timestamp#12, timestampType#13], StreamingRelation DataSource(org.apache.spark.sql.SparkSession@849355a,kafka,List(),None,List(),None,Map(kafka.bootstrap.servers -> kafka:9092, subscribe -> stations-json, startingOffsets -> earliest),None), kafka, [key#0, value#1, topic#2, partition#3, offset#4L, timestamp#5, timestampType#6]
